In [7]:
import os
os.environ['ZIPLINE_ROOT'] = os.path.join(os.getcwd(), '.zipline')
os.listdir(os.environ['ZIPLINE_ROOT'])
os.environ['ZIPLINE_TRADER_CONFIG'] = os.path.join(os.getcwd(), "./alpaca.yaml")

with open(os.environ['ZIPLINE_TRADER_CONFIG'], 'r') as f:
    data = f.read()
    print(data[:20])

# root zipline-trade


In [8]:
# load the yaml file into notebook 
# check if it is ok so we can connect to api
# !cat alpaca.yaml

# Ingest Bundle 

In [9]:
# alpaca_api bundle 
# start date is: 2015-12-31
# to adjust the start date go to the alpaca_api.py file 
# to adjust the universe go to the yaml file
!python zipline-trader/zipline/data/bundles/alpaca_api.py -f

zipline-trader/zipline/data/bundles/alpaca_api.py:278: UserWarning: Overwriting bundle with name 'alpaca_api'
  def api_to_bundle(interval=['1m']):
zipline-trader/zipline/data/bundles/alpaca_api.py:355: UserWarning: Overwriting bundle with name 'alpaca_api'
  end_session=end_date
Merging daily equity files:  [####################################]       
       symbol                 start_date  ... exchange asset_name
sid                                       ...                    
1        ACXP  2019-12-30 00:00:00+00:00  ...     NYSE       None
4        AEFC  2019-12-30 00:00:00+00:00  ...     NYSE       None
5       AGNCN  2019-12-30 00:00:00+00:00  ...     NYSE       None
7        ALGT  2019-12-30 00:00:00+00:00  ...     NYSE       None
8        APGB  2019-12-30 00:00:00+00:00  ...     NYSE       None
...       ...                        ...  ...      ...        ...
29771    VNTR  2019-12-30 00:00:00+00:00  ...     NYSE       None
29773  VYGG.U  2019-12-30 00:00:00+00:00  ...     

# Load your Bundle

In [10]:
import zipline
from zipline.data import bundles

bundle_name = 'alpaca_api'
bundle_data = bundles.load(bundle_name)

# Create The Data Portal

In [11]:
from zipline.pipeline.loaders import USEquityPricingLoader
from zipline.utils.calendars import get_calendar
from zipline.pipeline.data import USEquityPricing
from zipline.data.data_portal import DataPortal

import pandas as pd

# Set the dataloader
pricing_loader = USEquityPricingLoader.without_fx(bundle_data.equity_daily_bar_reader, bundle_data.adjustment_reader)


# Define the function for the get_loader parameter
def choose_loader(column):
    if column not in USEquityPricing.columns:
        raise Exception('Column not in USEquityPricing')
    return pricing_loader

# Set the trading calendar
trading_calendar = get_calendar('NYSE')

start_date = pd.Timestamp('2019-11-11', tz='utc')
end_date = pd.Timestamp('2021-12-23', tz='utc')

# Create a data portal
data_portal = DataPortal(bundle_data.asset_finder,
                         trading_calendar = trading_calendar,
                         first_trading_day = start_date,
                         equity_daily_reader = bundle_data.equity_daily_bar_reader,
                         adjustment_reader = bundle_data.adjustment_reader)

# Get Some Historical Data

In [12]:
equity = bundle_data.asset_finder.lookup_symbol("AAPL", end_date)
data_portal.get_history_window(assets=[equity], end_dt=end_date, bar_count=100,
                               frequency='1d',
                               field='close',
                               data_frequency='daily')


,Equity(6000 [AAPL])
2021-08-04 00:00:00+00:00,146.94
2021-08-05 00:00:00+00:00,147.06
2021-08-06 00:00:00+00:00,146.14
2021-08-09 00:00:00+00:00,146.09
2021-08-10 00:00:00+00:00,145.61
...,...
2021-12-17 00:00:00+00:00,170.86
2021-12-20 00:00:00+00:00,169.88
2021-12-21 00:00:00+00:00,172.98
2021-12-22 00:00:00+00:00,175.63
